In [ ]:
import pandas as pd
import networkx as nx
import string
import numpy as np
import scipy
from collections import Counter
from tabulate import tabulate
import matplotlib.pyplot as plt
import random
import json
import re
from tqdm import tqdm

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Load Functions

Load all the functions and the dictionary scraped in Scraping.ipynb

In [ ]:
with open('/content/drive/MyDrive/DDMCS/Translator_Long_Short_Names.json', 'r') as file:
    # Caricamento del contenuto del file in un dizionario
    Translator_LS = json.load(file)

bayern = Translator_LS.pop('B. Leverkusen')
Translator_LS['Bay. Leverkusen'] = bayern
Translator_LS['Stade Rennais']= 'Stade Rennais FC'    

In [ ]:
with open('/content/drive/MyDrive/DDMCS/MetaTranslatorV2.json', 'r') as file:
    # Caricamento del contenuto del file in un dizionario
   META_Translator = json.load(file)

In [ ]:
with open('/content/drive/MyDrive/DDMCS/dict_EUTOT.json', 'r') as file:
    # Caricamento del contenuto del file in un dizionario
    my_dict = json.load(file)

In [ ]:
with open('/content/drive/MyDrive/DDMCS/dict_AmericheTOT.json', 'r') as file:
    # Caricamento del contenuto del file in un dizionario
    my_dict_SATOT = json.load(file)

In [ ]:
with open('/content/drive/MyDrive/DDMCS/dict_ASIA.json', 'r') as file:
    # Caricamento del contenuto del file in un dizionario
    my_dict_ASIA = json.load(file)

In [ ]:
with open('/content/drive/MyDrive/DDMCS/dict_Africa.json', 'r') as file:
    # Caricamento del contenuto del file in un dizionario
    my_dict_Africa = json.load(file)

In [ ]:
with open('/content/drive/MyDrive/DDMCS/dict_FineTuning.json', 'r') as file:
    # Caricamento del contenuto del file in un dizionario
    dict_FineTuning = json.load(file)

In [ ]:
def FineTuning(value):
  Right_Names = df['club_name'].isin(df['club_involved_name']) # all the name that already match
  LS_DONTCHANGE=np.unique(df[Right_Names]['club_name'].tolist()).tolist() # store all the unique names that already match  

  for key, items in dict_FineTuning.items():
    if value in key:
      if value in LS_DONTCHANGE:
        return value
      else: 
        return items 
  return value      
  

In [ ]:
def TransaltionLS(value):
  stringaa='/content/drive/MyDrive/DDMCS/Long_Short_Translators/' + str(index_for_LS) +'.json'
  with open(stringaa, 'r') as file:
    # Caricamento del contenuto del file in un dizionario
    Translator_LS = json.load(file)
  bayern = Translator_LS.pop('B. Leverkusen')
  Translator_LS['Bay. Leverkusen'] = bayern
  Translator_LS['Stade Rennais']= 'Stade Rennais FC'


  Right_Names = df['club_name'].isin(df['club_involved_name']) # all the name that already match
  LS_DONTCHANGE=np.unique(df[Right_Names]['club_name'].tolist()).tolist() # store all the unique names that already match
  
  for key, items in Translator_LS.items():
    if value in items:
      if value in LS_DONTCHANGE:
        return value
      else: 
        return key  
  return value

In [ ]:
def get_key_Final(value):
    for key, items in my_dict.items():
        if value in items:
            return key
    return value

In [ ]:
def get_key_Nations(value):
    for key, items in META_Translator.items():
        if value in key:
            return items
    return value    

In [ ]:
def apply_Trasformations(df):
  #df.loc[:,'club_involved_name'] = df['club_involved_name'].map(FineTuning) # change only the right names (when we talk about the U23 and so on)
  df.loc[:,'club_name'] = df['club_name'].map(TransaltionLS) # change only the right names
  mask = ~df['club_involved_name'].isin(df['club_name']) # select only the name that are not in the first column
  df_copy=df[mask]

  # Stacking all my dictionary togheter 
  my_dict.update(my_dict_SATOT)
  my_dict.update(my_dict_ASIA)
  my_dict.update(my_dict_Africa)

  df_copy_2 = df_copy.copy()
  df_copy_2.loc[:,'club_involved_name'] = df_copy['club_involved_name'].map(get_key_Final) # Translate the name of the less important club into the name of their leagues
  df_copy_2.loc[:,'club_involved_name'] = df_copy_2['club_involved_name'].map(get_key_Nations) # Translate each leagues with META-Translator (ex: 'BE1' To 'Belgium')


############### TRY TO RESCUE SOME NAMES AGAIN ######################################################################


  # Apply the dictionary in which I try another time to rescue the Team Name (when a club have the name U23 and so on)
  df_copy_2.loc[:,'club_involved_name'] = df_copy_2['club_involved_name'].map(FineTuning)

  mask2 = ~df_copy_2['club_involved_name'].isin(df_copy_2['club_name']) # select only the name that are not in the first column
  df_copy3=df_copy_2[mask2]

  df_copy_4 = df_copy3.copy()
  df_copy_4.loc[:,'club_involved_name'] = df_copy3['club_involved_name'].map(get_key_Final) # Translate the name of the less important club into the name of their leagues
  df_copy_4.loc[:,'club_involved_name'] = df_copy_4['club_involved_name'].map(get_key_Nations) # Translate each leagues with META-Translator (ex: 'BE1' To 'Belgium')

  # Re-connect mask 2
  df_copy_2.loc[df_copy_2.index.isin(df_copy_4.index)] = df_copy_4

###################################################################################################################

  # Re-connect mask 
  df.loc[df.index.isin(df_copy_2.index)] = df_copy_2
  return df, df_copy_2


In [ ]:
def efficiencyTranslator(df_copy, scarti):
  mask = ~df_copy['club_involved_name'].isin(df_copy['club_name'])
  df_ss=df[mask]
  raw_list=np.unique(df_ss['club_involved_name']).tolist()
  check_list= [v for k, v in list(META_Translator.items())]
  check_list=np.unique(check_list).tolist()
  result = list(set(raw_list) - set(check_list))
  return len(result)

# Prova

load the total dataset

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/DDMCS/Out_For_Cleaning.csv', delimiter=';', encoding= 'unicode_escape')

In [ ]:
df.head()

,Unnamed: 0,club_name,player_name,age,position,club_involved_name,fee,transfer_movement,transfer_period,fee_cleaned,league_name,year,season
0,1,AC Fiorentina,Brian Laudrup,23.0,Right Winger,Bayern Munich,5.00m,in,Summer,5,Serie A,1992,1992/1993
1,2,AC Fiorentina,Stefan Effenberg,23.0,Central Midfield,Bayern Munich,3.75m,in,Summer,"3,75",Serie A,1992,1992/1993
2,3,AC Fiorentina,Daniele Carnasciali,25.0,Right-Back,Brescia,?,in,Summer,NaN,Serie A,1992,1992/1993
3,4,AC Fiorentina,Diego Latorre,22.0,Second Striker,Boca Juniors,?,in,Summer,NaN,Serie A,1992,1992/1993
4,5,AC Fiorentina,Andrea Vascotto,20.0,midfield,Barletta CS,?,in,Summer,NaN,Serie A,1992,1992/1993


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 132725 entries, 0 to 132724
Data columns (total 13 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   Unnamed: 0          132725 non-null  int64  
 1   club_name           132725 non-null  object 
 2   player_name         132725 non-null  object 
 3   age                 132671 non-null  float64
 4   position            132725 non-null  object 
 5   club_involved_name  132725 non-null  object 
 6   fee                 132640 non-null  object 
 7   transfer_movement   132725 non-null  object 
 8   transfer_period     132725 non-null  object 
 9   fee_cleaned         51040 non-null   object 
 10  league_name         132725 non-null  object 
 11  year                132725 non-null  int64  
 12  season              132725 non-null  object 
dtypes: float64(1), int64(2), object(10)
memory usage: 13.2+ MB


 First operations of preprocessing

In [ ]:
df = df.drop("Unnamed: 0", axis=1) # Removing The first column that is useless

df = df[df["transfer_movement"] != "out"] # Removing all the out movement 

df = df[df["fee"] != "-"] # Removing all the movement in which a club buy a player from the Youth accademy of itself

df.loc[df["fee"] == "free transfer", "club_involved_name"] = "free agents" # Classify all the free agents movements with the right names

# Remove all the returns from loans
df['fee'] = df['fee'].astype(str) 
df = df[~df['fee'].str.startswith("End of")]

# Change '?' with 0.01
df['fee_cleaned'].fillna(0.01, inplace=True)


In [ ]:
df.head()

,club_name,player_name,age,position,club_involved_name,fee,transfer_movement,transfer_period,fee_cleaned,league_name,year,season
0,AC Fiorentina,Brian Laudrup,23.0,Right Winger,Bayern Munich,5.00m,in,Summer,5,Serie A,1992,1992/1993
1,AC Fiorentina,Stefan Effenberg,23.0,Central Midfield,Bayern Munich,3.75m,in,Summer,"3,75",Serie A,1992,1992/1993
2,AC Fiorentina,Daniele Carnasciali,25.0,Right-Back,Brescia,?,in,Summer,0.01,Serie A,1992,1992/1993
3,AC Fiorentina,Diego Latorre,22.0,Second Striker,Boca Juniors,?,in,Summer,0.01,Serie A,1992,1992/1993
4,AC Fiorentina,Andrea Vascotto,20.0,midfield,Barletta CS,?,in,Summer,0.01,Serie A,1992,1992/1993


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 40034 entries, 0 to 132717
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   club_name           40034 non-null  object 
 1   player_name         40034 non-null  object 
 2   age                 40021 non-null  float64
 3   position            40034 non-null  object 
 4   club_involved_name  40034 non-null  object 
 5   fee                 40034 non-null  object 
 6   transfer_movement   40034 non-null  object 
 7   transfer_period     40034 non-null  object 
 8   fee_cleaned         40034 non-null  object 
 9   league_name         40034 non-null  object 
 10  year                40034 non-null  int64  
 11  season              40034 non-null  object 
dtypes: float64(1), int64(1), object(10)
memory usage: 4.0+ MB


Seasons names

In [ ]:
colonna_values = df['season'].unique().tolist()

Apply the Changing in all my datasets

In [ ]:
for value in tqdm(colonna_values):  

    #loss=[] # Initialize the loss
    index_for_LS = value.split('/')[0] # Took the index


    # Seleziona i dati in base al valore corrente
    mini_df = df[df['season'] == value]
    df_copy, scarti = apply_Trasformations(mini_df)
    #loss.append(efficiencyTranslator(df_copy, scarti))
    # Create the Meta-Translator
    string= '/content/drive/MyDrive/DDMCS/Cleaned_datasetsV3/' + str(value).replace("/", "_") +'.csv'
    df_copy.to_csv(string, index=False)

    if 'mini_df' in locals() or 'mini_df' in globals():
      del mini_df

100%|██████████| 30/30 [17:11<00:00, 34.38s/it]
